In [1]:
import os
import ssl
import time
import socket
import requests
import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from urllib import parse
from tqdm.notebook import tqdm

from bs4 import BeautifulSoup
from fake_useragent import UserAgent

from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager #!pip install webdriver-manager

# 한글대응
from matplotlib import rc
%matplotlib inline
plt.rcParams['axes.unicode_minus'] = False
rc('font',family='Malgun Gothic')

ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
def create_folder(directory):
    import os
    
    if not os.path.exists(directory):
        os.mkdir(directory)

In [3]:
def yogiyo_crawling(get_review_num, save_dir, menu='치킨'):
    # 사이트 접속
    url = 'https://www.yogiyo.co.kr/mobile/#/'
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.get(url); action = ActionChains(driver); time.sleep(1)
    driver.maximize_window(); time.sleep(1)
    
    ##----------- 지정 주소 검색 -----------
    search_addr = driver.find_element(By.CSS_SELECTOR, '#search > div > form > input')
    search_addr.send_keys(Keys.CONTROL + "A")
    search_addr.send_keys("서초대로38길 12, 마제스타시티")

    driver.find_element(By.CSS_SELECTOR, "#button_search_address > button.btn.btn-default.ico-pick").click()
    time.sleep(1)
    driver.find_element(By.CSS_SELECTOR, "#search > div > form > ul > li:nth-child(3) > a").click()
    time.sleep(1)

    ##----------- 메뉴 검색 -----------
    # 돋보기 클릭
    driver.find_element(By.CSS_SELECTOR, "#category > ul > li.hidden-xs.menu-search > a").click()
    time.sleep(1)

    search_menu = driver.find_element(By.CSS_SELECTOR, "#category > ul > li.main-search > form > div > input")
    search_menu.send_keys(menu)
    time.sleep(1)
    
    driver.find_element(By.CSS_SELECTOR, "#category_search_button").click()
    time.sleep(1)

    ##----------- 가게 검색 -----------
    #가게이름 클릭
    driver.find_element(By.CSS_SELECTOR, "#content > div > div:nth-child(5) > div > div > div > div").click()
    time.sleep(1)
    
    # 리뷰탭 클릭
    driver.find_element(By.CSS_SELECTOR, '#content > div.restaurant-detail.row.ng-scope > div.col-sm-8 > ul > li:nth-child(2) > a').click()
    time.sleep(1)
    
    # 사진 리뷰만 보기
    driver.find_element(By.CSS_SELECTOR, '#content > div.restaurant-detail.row.ng-scope > div.col-sm-8 > div:nth-child(6) > div.review-count > div > div > div > label').click()
    time.sleep(1)
    
    
    ##----------- 이미지 크롤링  -----------
    # 스크롤 맨 아래로
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);"); time.sleep(1)  # 브라우저 끝까지 스크롤을 내림
    
    review_num = len(driver.find_elements(By.XPATH, '//*[@id="review"]/li'))
    
    while (review_num < get_review_num):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")  # 브라우저 끝까지 스크롤을 내림
        time.sleep(1)
        driver.find_element(By.CSS_SELECTOR, '#review > li.list-group-item.btn-more > a').click()
        time.sleep(1)
        review_num = len(driver.find_elements(By.XPATH, '//*[@id="review"]/li'))
        
    # 현재 화면의 html 코드를 가져와서 뷰숲으로 작업하기
    req = driver.page_source
    soup = BeautifulSoup(req, 'html.parser')
    
    # 리뷰 목록 불러오기
    review_tables = soup.select('#review>li')[1:get_review_num+1]
    
    make_save_folder = save_dir + 'reviews_data'
    create_folder(make_save_folder)

    for n, review in enumerate(review_tables):
        img_tag = review.select('table>tbody>tr>td>div>img')
        image_urls = [each['data-url'] for each in img_tag]

        make_user_folder = make_save_folder+'/'+menu+'reviewer'+str(n+1)
        create_folder(make_user_folder)

        for idx, url in enumerate(image_urls):
            response = requests.get(url)

            if response.status_code == 200:
                image_file = os.path.join(make_user_folder, f'img{idx+1}.jpg')
                with open(image_file, 'wb') as file:
                    file.write(response.content)
                #print('이미지 저장', image_file)
            else:
                print('이미지 저장 실패')
    
    print(f'[{menu}] 크롤링 완료')
    driver.quit()

In [4]:
my_menu = ['짜장면', '짬뽕', '탕수육']

for menu in my_menu:
    yogiyo_crawling(get_review_num=100, save_dir='../Data_total/', menu=menu)

[짜장면] 크롤링 완료
[짬뽕] 크롤링 완료
[탕수육] 크롤링 완료
